In [181]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import joblib
from scipy.stats import zscore
from sklearn.preprocessing import LabelEncoder

In [182]:
df_train = pd.read_csv(r'DataSets\train.csv')
df_test = pd.read_csv(r'DataSets\test.csv')
df_holdout = pd.read_csv(r'DataSets\holdout.csv')

step 1) Train Dataset understanding

In [183]:
df_train.head(10)

,company_id,company_name,industry,funding_rounds,total_funding,last_funding_date,job_postings_30d,employee_growth_pct,hiring_roles,industry_growth_rate,regional_employment_trend,funding_per_employee,days_since_last_funding,growth_momentum,is_hot_lead
0,COMP_024008,Moreno and Sons,Retail,1.911533,3795597.04,2021-07-14,10.995177,-8.355216,Operations,11.5,-0.2,-5930.620375,1317,45547164.48,0
1,COMP_007976,"Morales, Hubbard and Mcdonald",Healthcare,3.221005,5792195.29,2020-05-23,5.052385,20.602718,Marketing,23.9,10.5,2106.252833,1734,34753171.74,0
2,COMP_021959,Strickland-Wright,Tech,3.975966,191991.00,2020-05-07,5.008945,0.456013,Operations,20.5,-1.5,1371.364286,1750,959955.00,0
3,COMP_012658,Sanders Group,Fintech,2.842382,890042.57,2023-04-11,9.882709,1.202142,Operations,23.2,13.9,4238.297952,681,8010383.13,0
4,COMP_011887,Davis Inc,Retail,2.264359,3369725.77,2024-03-27,11.461475,22.633126,Marketing,9.0,8.6,1567.314312,330,40436709.24,1
5,COMP_024359,Underwood-Moore,Healthcare,4.875184,2881335.43,2021-09-17,6.869437,32.667976,Marketing,21.0,1.0,752.306901,1252,20169348.01,0
6,COMP_010563,"Mendez, Walker and Garcia",Fintech,0.900992,7941005.00,2022-03-03,7.892017,22.698968,Marketing,14.5,-3.6,3214.981781,1085,63528040.00,0
7,COMP_007167,Dennis-Hicks,Fintech,1.170316,1716573.56,2021-03-16,11.931091,8.556498,Engineering,29.4,-1.2,1928.734337,1437,24032029.84,0
8,COMP_014578,House and Sons,SaaS,3.337948,178278.14,2024-08-31,7.100843,30.058273,Engineering,25.2,10.4,52.745012,173,1426225.12,0
9,COMP_015475,"Neal, Fernandez and Montgomery",Tech,3.864427,2266813.43,2024-12-07,9.548882,-2.022682,Engineering,14.4,9.2,-25186.815889,75,27201761.16,0


In [184]:
Rows = df_train.shape[0]
columns = df_train.shape[1]
print(f"Number of Rows : {Rows}")
print(f"Number of Columns : {columns}")

Number of Rows : 20000
Number of Columns : 15


In [185]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 15 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   company_id                 20000 non-null  object 
 1   company_name               20000 non-null  object 
 2   industry                   20000 non-null  object 
 3   funding_rounds             20000 non-null  float64
 4   total_funding              20000 non-null  float64
 5   last_funding_date          20000 non-null  object 
 6   job_postings_30d           20000 non-null  float64
 7   employee_growth_pct        20000 non-null  float64
 8   hiring_roles               20000 non-null  object 
 9   industry_growth_rate       20000 non-null  float64
 10  regional_employment_trend  20000 non-null  float64
 11  funding_per_employee       20000 non-null  float64
 12  days_since_last_funding    20000 non-null  int64  
 13  growth_momentum            20000 non-null  flo

In [186]:
df_train.describe()

d:\DataScience\GUVI\DataScience_GUVI_Projects\Customer_Conversion_Analysis_MLOps\customer_Convesion_Analysis_VENV\lib\site-packages\pandas\core\nanops.py:1016: RuntimeWarning: invalid value encountered in subtract
  sqr = _ensure_numeric((avg - values) ** 2)


,funding_rounds,total_funding,job_postings_30d,employee_growth_pct,industry_growth_rate,regional_employment_trend,funding_per_employee,days_since_last_funding,growth_momentum,is_hot_lead
count,20000.000000,2.000000e+04,20000.000000,20000.000000,20000.000000,20000.000000,2.000000e+04,20000.000000,2.000000e+04,20000.000000
mean,2.987775,4.171424e+06,10.002013,19.954379,17.456325,5.133810,inf,909.826000,4.188535e+07,0.149250
std,1.454144,3.088695e+06,3.328233,17.532040,7.232038,5.767425,NaN,527.341786,3.525597e+07,0.356344
min,0.686082,1.004476e+05,0.926065,-12.491863,5.000000,-5.000000,-7.894026e+05,1.000000,2.957271e+05,0.000000
25%,1.954142,1.719901e+06,7.640501,4.824062,11.200000,0.200000,3.183275e+02,455.000000,1.532106e+07,0.000000
50%,2.869536,3.533375e+06,9.733120,19.865427,17.400000,5.200000,1.181334e+03,905.000000,3.279494e+07,0.000000
75%,3.868760,5.976700e+06,12.056417,34.704816,23.700000,10.200000,2.786092e+03,1363.000000,5.888261e+07,0.000000
max,13.013271,2.141484e+07,29.457835,64.173316,30.000000,15.000000,inf,1827.000000,3.239826e+08,1.000000


step 2) Null Values Check

In [187]:
#Checking For Null Values in the Dataset
df_train.isna().sum()

company_id                   0
company_name                 0
industry                     0
funding_rounds               0
total_funding                0
last_funding_date            0
job_postings_30d             0
employee_growth_pct          0
hiring_roles                 0
industry_growth_rate         0
regional_employment_trend    0
funding_per_employee         0
days_since_last_funding      0
growth_momentum              0
is_hot_lead                  0
dtype: int64

step 3) Check Duplicate values

In [188]:
#check for duplicate values in the dataset
df_train.duplicated().sum()

0

step 4) Check for unique values

In [189]:
#Function definition for checking Nunique values
def check_unique_values(columns:list):
    for col in columns:
        print(f"{col} : {df_train[col].nunique()}")
        print("------------------------------")

In [190]:
continuos_cols = df_train.select_dtypes(include=["int","float"]).columns.to_list()

#call Function for Nunique value check
print("Continuos Columns\n")
check_unique_values(continuos_cols)


Continuos Columns

funding_rounds : 20000
------------------------------
total_funding : 20000
------------------------------
job_postings_30d : 20000
------------------------------
employee_growth_pct : 19973
------------------------------
industry_growth_rate : 251
------------------------------
regional_employment_trend : 201
------------------------------
funding_per_employee : 19966
------------------------------
days_since_last_funding : 1827
------------------------------
growth_momentum : 20000
------------------------------
is_hot_lead : 2
------------------------------


In [191]:
categorical_cols = df_train.select_dtypes(include=["object","category"]).columns.to_list()

#call Function for Nunique value check
print("Categorical Columns\n")
check_unique_values(categorical_cols)

Categorical Columns

company_id : 20000
------------------------------
company_name : 16366
------------------------------
industry : 5
------------------------------
last_funding_date : 1827
------------------------------
hiring_roles : 4
------------------------------


In [192]:
#Dropping the column 'company_id' from Train dataset
df_train.drop(columns=['company_id'],inplace= True)

#Dropping the column 'company_id from Test DataSet 
df_test.drop(columns=['company_id'],inplace= True)

# Set 'company_id' as the index column
df_holdout.set_index('company_id', inplace=True)

Key takeaway from the step 4: The "company_id" column contains only unique values, making it unsuitable for model training as it doesn't provide meaningful patterns or insights. Hence, we will exclude it from the model training.

step 5) Detecting and dropping the 'Inf' value in the 'funding_per_employee' column from the Train.csv, Test.csv, holdout.csv datasets

In [193]:
def Handling_inf(df:pd.DataFrame , column: str):
    """
    Detecting and dropping the inf values in the column from all the datatests

    Args:
    Df --> Pandas Dataframe
    column --> Column name for searching inf values

    Return: Inf value Handled Pandas DataFrame 

    """

    # Count the number of 'inf' values
    inf_count = df[column].isin([np.inf, -np.inf]).sum()

    if inf_count > 0:
        df[column] = df[column].replace([np.inf, -np.inf], np.nan)  # Replace inf with NaN
        df.dropna(subset=[column], inplace=True)  # Drop rows where column has NaN
    else:
        pass

    return df , inf_count


In [194]:
column = "funding_per_employee"

In [195]:
# Train Data 'inf' value handle
df_train, inf_count = Handling_inf(df_train, column)

# Print the count of inf values
print(f"Number of 'inf' values in '{column}': {inf_count}")

#Print the Size of the Dataset after dropping 'inf' values 
Rows = df_train.shape[0]
columns = df_train.shape[1]
print(f"Number of Rows : {Rows}")
print(f"Number of Columns : {column}")

Number of 'inf' values in 'funding_per_employee': 35
Number of Rows : 19965
Number of Columns : funding_per_employee


In [196]:
# Test Data 'inf' value handle
df_test, inf_count = Handling_inf(df_test, column)

# Print the count of inf values
print(f"Number of 'inf' values in '{column}': {inf_count}")

#Print the Size of the Dataset after dropping 'inf' values 
Rows = df_test.shape[0]
columns = df_test.shape[1]
print(f"Number of Rows : {Rows}")
print(f"Number of Columns : {column}")

Number of 'inf' values in 'funding_per_employee': 8
Number of Rows : 4992
Number of Columns : funding_per_employee


In [197]:
# Holdout Data 'inf' value handle
df_holdout, inf_count = Handling_inf(df_holdout, column)

# Print the count of inf values
print(f"Number of 'inf' values in '{column}': {inf_count}")

#Print the Size of the Dataset after dropping 'inf' values 
Rows = df_holdout.shape[0]
columns = df_holdout.shape[1]
print(f"Number of Rows : {Rows}")
print(f"Number of Columns : {column}")

Number of 'inf' values in 'funding_per_employee': 2
Number of Rows : 498
Number of Columns : funding_per_employee


Key take away from this step 5 : The inf values in the 'funding_per_employee' column were detected (35 in train data) and successfully removed, reducing the dataset to 19,965 rows. The same process should be applied to the test and holdout datasets for consistency before model training.

Step 6) Data Type Conversion

In [198]:
def DataTime_conversion(df:pd.DataFrame,column:str):
    
    """
    Conversion of the column dtype to DateTime

    Args:
    df --> Pandas DataFrame
    column --> The column dtype need to be converted to the DateTime

    return --> Pandas DatFrame

    """
    df[column] = pd.to_datetime(df[column], errors='coerce')
    return df

In [199]:
column1 = "last_funding_date"

In [200]:
#calling function to Convert the dtype to DateTime Type
df_train = DataTime_conversion(df_train, column1)
print(df_train[column1].dtype)

datetime64[ns]


In [201]:
#calling function to Convert the dtype to DateTime Type
df_test = DataTime_conversion(df_test, column1)
print(df_test[column1].dtype)

datetime64[ns]


In [202]:
#calling function to Convert the dtype to DateTime Type
df_holdout = DataTime_conversion(df_holdout, column1)
print(df_holdout[column1].dtype)

datetime64[ns]


Key take away from this step 6: The last_funding_date column was successfully converted to datetime64[ns] format using pd.to_datetime(), ensuring proper handling of date-based features. This transformation enables further feature extraction for model training.

Step 7) Outliers Detecting and Handling

In [203]:
# Function to calculate Z-score bounds and outlier counts
def detect_outliers_zscore(df:pd.DataFrame, columns:list, threshold=3):
    outlier_stats = []

    if 'is_hot_lead' in columns:
        columns.remove('is_hot_lead')

    for col in columns:
        # Calculate Z-scores using scipy's zscore method
        z_scores = zscore(df[col])
        
        # Count outliers based on the threshold
        outliers = (abs(z_scores) > threshold).sum()
        
        # Append results for this column
        outlier_stats.append({
            "Column": col,
            "Outliers": outliers
        })
    
    return pd.DataFrame(outlier_stats)

In [204]:
# Detect outliers using Z-score for df_train
outlier_zscore_stats = detect_outliers_zscore(df_train, continuos_cols)

# Display the Z-score outlier statistics
outlier_zscore_stats

,Column,Outliers
0,funding_rounds,173
1,total_funding,174
2,job_postings_30d,90
3,employee_growth_pct,0
4,industry_growth_rate,0
5,regional_employment_trend,0
6,funding_per_employee,234
7,days_since_last_funding,0
8,growth_momentum,298


key take away from this step 7 : Since the detected outliers are not technically outliers based on domain knowledge, we will retain them without applying any handling techniques. This ensures that valuable patterns in the data are preserved for model training.

step 8) Feature Engineering 

In [206]:
def Feature_Engineering(df: pd.DataFrame):
    """
    Applying Feature Engineering to the datas

    Args:
    df --> Pandas DataFrame

    Return:
    pandas DataFrame

    """

    df['funding_year'] = df['last_funding_date'].dt.year
    df['funding_month'] = df['last_funding_date'].dt.month
    df['funding_day'] = df['last_funding_date'].dt.day
    df['funding_weekday'] = df['last_funding_date'].dt.weekday  
    df['funding_quarter'] = df['last_funding_date'].dt.quarter
    df.drop(columns= ['last_funding_date'],inplace= True)

    return df


In [207]:
df_train = Feature_Engineering(df_train)

In [208]:
df_test = Feature_Engineering(df_test)

In [209]:
df_holdout = Feature_Engineering(df_holdout)

In [210]:
df_train.head(10)

,company_name,industry,funding_rounds,total_funding,job_postings_30d,employee_growth_pct,hiring_roles,industry_growth_rate,regional_employment_trend,funding_per_employee,days_since_last_funding,growth_momentum,is_hot_lead,funding_year,funding_month,funding_day,funding_weekday,funding_quarter
0,Moreno and Sons,Retail,1.911533,3795597.04,10.995177,-8.355216,Operations,11.5,-0.2,-5930.620375,1317,45547164.48,0,2021,7,14,2,3
1,"Morales, Hubbard and Mcdonald",Healthcare,3.221005,5792195.29,5.052385,20.602718,Marketing,23.9,10.5,2106.252833,1734,34753171.74,0,2020,5,23,5,2
2,Strickland-Wright,Tech,3.975966,191991.00,5.008945,0.456013,Operations,20.5,-1.5,1371.364286,1750,959955.00,0,2020,5,7,3,2
3,Sanders Group,Fintech,2.842382,890042.57,9.882709,1.202142,Operations,23.2,13.9,4238.297952,681,8010383.13,0,2023,4,11,1,2
4,Davis Inc,Retail,2.264359,3369725.77,11.461475,22.633126,Marketing,9.0,8.6,1567.314312,330,40436709.24,1,2024,3,27,2,1
5,Underwood-Moore,Healthcare,4.875184,2881335.43,6.869437,32.667976,Marketing,21.0,1.0,752.306901,1252,20169348.01,0,2021,9,17,4,3
6,"Mendez, Walker and Garcia",Fintech,0.900992,7941005.00,7.892017,22.698968,Marketing,14.5,-3.6,3214.981781,1085,63528040.00,0,2022,3,3,3,1
7,Dennis-Hicks,Fintech,1.170316,1716573.56,11.931091,8.556498,Engineering,29.4,-1.2,1928.734337,1437,24032029.84,0,2021,3,16,1,1
8,House and Sons,SaaS,3.337948,178278.14,7.100843,30.058273,Engineering,25.2,10.4,52.745012,173,1426225.12,0,2024,8,31,5,3
9,"Neal, Fernandez and Montgomery",Tech,3.864427,2266813.43,9.548882,-2.022682,Engineering,14.4,9.2,-25186.815889,75,27201761.16,0,2024,12,7,5,4


key take away from this step 8: The Feature Engineering function was implemented to extract meaningful insights from the dataset, enhancing predictive power. The df_train dataset was successfully transformed with new features, preparing it for model training. 

step 9) encoding the categorical Columns

In [211]:
# Store Label Encoders & Encoded Mappings
label_encoders_dict = {}
category_encoding_map = {}

# Encode categorical columns in training data
for category_column in df_train.select_dtypes(include=['object']).columns:
    encoder = LabelEncoder()
    df_train[category_column] = encoder.fit_transform(df_train[category_column])
    label_encoders_dict[category_column] = encoder  # Store the encoder
    category_encoding_map[category_column] = dict(zip(encoder.classes_, encoder.transform(encoder.classes_)))  # Store mappings


In [212]:
# Apply encoding to test and holdout datasets
for dataset in [df_test, df_holdout]:
    for category_column in dataset.select_dtypes(include=['object']).columns:
        if category_column in label_encoders_dict:
            dataset[category_column] = dataset[category_column].map(
                lambda value: label_encoders_dict[category_column].transform([value])[0] 
                if value in label_encoders_dict[category_column].classes_ else -1
            )

In [213]:
# Handle missing values by replacing NaNs with -1
for dataset in [df_train, df_test, df_holdout]:
    if dataset.isna().sum().sum() != 0:
        dataset.fillna(-1, inplace=True)

In [214]:
df_test.describe()

,company_name,industry,funding_rounds,total_funding,job_postings_30d,employee_growth_pct,hiring_roles,industry_growth_rate,regional_employment_trend,funding_per_employee,days_since_last_funding,growth_momentum,is_hot_lead,funding_year,funding_month,funding_day,funding_weekday,funding_quarter
count,4992.000000,4992.000000,4992.000000,4.992000e+03,4992.000000,4992.000000,4992.000000,4992.000000,4992.000000,4.992000e+03,4992.000000,4.992000e+03,4992.000000,4992.000000,4992.000000,4992.000000,4992.000000,4992.000000
mean,2214.144231,2.007212,3.028847,4.196975e+06,9.992610,19.863740,1.503005,17.444812,4.875761,4.639143e+02,916.082732,4.211427e+07,0.149439,2022.127204,6.567107,15.759615,3.007212,2.523037
std,4374.603774,1.405811,1.470555,3.074335e+06,3.361154,17.692882,1.108966,7.180686,5.755938,4.816808e+04,529.540827,3.525469e+07,0.356556,1.462428,3.460905,8.790597,2.001890,1.119342
min,-1.000000,0.000000,0.638243,1.021974e+05,0.824309,-12.026707,0.000000,5.000000,-5.000000,-1.389559e+06,1.000000,5.831372e+05,0.000000,2020.000000,1.000000,1.000000,0.000000,1.000000
25%,-1.000000,1.000000,1.972658,1.769002e+06,7.640035,4.397746,1.000000,11.200000,0.000000,3.142901e+02,455.750000,1.539638e+07,0.000000,2021.000000,4.000000,8.000000,1.000000,2.000000
50%,-1.000000,2.000000,2.883502,3.526123e+06,9.763089,19.711660,1.000000,17.600000,4.800000,1.169295e+03,919.500000,3.313974e+07,0.000000,2022.000000,7.000000,16.000000,3.000000,3.000000
75%,1458.500000,3.000000,3.937650,6.094331e+06,12.099687,34.515293,2.000000,23.500000,9.800000,2.841203e+03,1367.000000,5.890179e+07,0.000000,2023.000000,10.000000,23.000000,5.000000,4.000000
max,16326.000000,4.000000,9.712037,1.966398e+07,27.240139,64.910738,3.000000,30.000000,15.000000,1.069657e+06,1827.000000,2.600835e+08,1.000000,2025.000000,12.000000,31.000000,6.000000,4.000000


In [216]:
df_holdout.describe()

,company_name,industry,funding_rounds,total_funding,job_postings_30d,employee_growth_pct,hiring_roles,industry_growth_rate,regional_employment_trend,funding_per_employee,days_since_last_funding,growth_momentum,funding_year,funding_month,funding_day,funding_weekday,funding_quarter
count,498.000000,498.000000,498.000000,4.980000e+02,498.000000,498.000000,498.000000,498.000000,498.000000,498.000000,498.000000,4.980000e+02,498.000000,498.000000,498.000000,498.000000,498.000000
mean,1960.791165,2.058233,3.110442,4.118849e+06,9.726908,20.677510,1.578313,17.439157,4.848594,971.644017,908.676707,4.121133e+07,2022.142570,6.620482,15.985944,2.861446,2.518072
std,4176.186818,1.425064,1.409172,2.927568e+06,3.152993,17.291727,1.167506,7.213606,5.784153,42600.759002,511.020990,3.474048e+07,1.396228,3.499719,8.666281,1.997705,1.138619
min,-1.000000,0.000000,1.000000,1.073545e+05,1.000000,-9.600000,0.000000,5.000000,-5.000000,-416600.634000,2.000000,3.777140e+05,2020.000000,1.000000,1.000000,0.000000,1.000000
25%,-1.000000,1.000000,2.000000,1.740622e+06,8.000000,5.800000,0.000000,11.425000,-0.500000,357.761146,485.250000,1.456593e+07,2021.000000,4.000000,9.000000,1.000000,2.000000
50%,-1.000000,2.000000,3.000000,3.595327e+06,10.000000,21.450000,2.000000,17.700000,4.900000,1140.484994,872.500000,3.138121e+07,2022.000000,6.000000,16.000000,3.000000,2.000000
75%,-1.000000,3.000000,4.000000,5.979098e+06,12.000000,35.875000,3.000000,23.875000,9.600000,2718.000973,1348.000000,6.116454e+07,2023.000000,10.000000,23.000000,5.000000,4.000000
max,16211.000000,4.000000,8.000000,1.521680e+07,20.000000,50.000000,3.000000,29.900000,15.000000,564662.337500,1823.000000,1.978184e+08,2025.000000,12.000000,31.000000,6.000000,4.000000


key take away from this step 9 : The categorical columns were successfully encoded using Label Encoding, converting text data into numerical values for model training. The encoded data was also saved using joblib, ensuring reusability for future predictions.

Step 10) Storing the Preprocessed data to local

In [217]:
# Processed Train Dataset to local
df_train.to_csv('Datasets/Processed_train_data.csv',index=False)
df_train.shape

(19965, 18)

In [218]:
df_train.head(10)

,company_name,industry,funding_rounds,total_funding,job_postings_30d,employee_growth_pct,hiring_roles,industry_growth_rate,regional_employment_trend,funding_per_employee,days_since_last_funding,growth_momentum,is_hot_lead,funding_year,funding_month,funding_day,funding_weekday,funding_quarter
0,9973,2,1.911533,3795597.04,10.995177,-8.355216,2,11.5,-0.2,-5930.620375,1317,45547164.48,0,2021,7,14,2,3
1,9938,1,3.221005,5792195.29,5.052385,20.602718,1,23.9,10.5,2106.252833,1734,34753171.74,0,2020,5,23,5,2
2,14014,4,3.975966,191991.00,5.008945,0.456013,2,20.5,-1.5,1371.364286,1750,959955.00,0,2020,5,7,3,2
3,12831,0,2.842382,890042.57,9.882709,1.202142,2,23.2,13.9,4238.297952,681,8010383.13,0,2023,4,11,1,2
4,3203,2,2.264359,3369725.77,11.461475,22.633126,1,9.0,8.6,1567.314312,330,40436709.24,1,2024,3,27,2,1
5,14637,1,4.875184,2881335.43,6.869437,32.667976,1,21.0,1.0,752.306901,1252,20169348.01,0,2021,9,17,4,3
6,9405,0,0.900992,7941005.00,7.892017,22.698968,1,14.5,-3.6,3214.981781,1085,63528040.00,0,2022,3,3,3,1
7,3444,0,1.170316,1716573.56,11.931091,8.556498,0,29.4,-1.2,1928.734337,1437,24032029.84,0,2021,3,16,1,1
8,6539,3,3.337948,178278.14,7.100843,30.058273,0,25.2,10.4,52.745012,173,1426225.12,0,2024,8,31,5,3
9,10332,4,3.864427,2266813.43,9.548882,-2.022682,0,14.4,9.2,-25186.815889,75,27201761.16,0,2024,12,7,5,4


In [219]:
df_train['is_hot_lead'].value_counts()

is_hot_lead
0    16980
1     2985
Name: count, dtype: int64

In [220]:
# Processed Test Dataset to local
df_test.to_csv('Datasets/Processed_test_data.csv',index=False)
df_test.shape

(4992, 18)

In [221]:
df_test['is_hot_lead'].value_counts()

is_hot_lead
0    4246
1     746
Name: count, dtype: int64

In [222]:
df_test.head(10)

,company_name,industry,funding_rounds,total_funding,job_postings_30d,employee_growth_pct,hiring_roles,industry_growth_rate,regional_employment_trend,funding_per_employee,days_since_last_funding,growth_momentum,is_hot_lead,funding_year,funding_month,funding_day,funding_weekday,funding_quarter
0,-1,2,3.265246,2425223.28,7.201083,5.569041,2,8.0,8.4,3566.504824,1280,16976562.96,0,2021,8,20,4,3
1,-1,3,1.912502,3988620.41,14.562457,46.348625,0,7.1,4.3,890.317056,1729,55840685.74,1,2020,5,28,3,2
2,-1,4,5.425373,2645234.46,14.797940,34.121940,1,16.0,14.8,669.679610,1449,37033282.44,0,2021,3,4,3,1
3,-1,2,0.987882,3194492.15,14.946632,4.969312,3,21.1,5.9,5414.393475,849,41528397.95,0,2022,10,25,1,4
4,-1,2,3.583654,884497.73,9.714369,-7.111839,2,8.1,-4.3,-1499.148695,1572,9729475.03,0,2020,11,1,6,4
5,-1,3,2.340446,6076529.42,12.142480,0.278950,3,6.7,0.9,46742.534000,216,72918353.04,0,2024,7,19,4,3
6,14877,1,3.421293,2256189.13,12.677383,23.829309,2,15.1,-4.8,951.978536,422,27074269.56,1,2023,12,26,1,4
7,10897,0,2.047862,6243040.88,10.374001,29.645961,3,11.9,12.3,2190.540660,1570,68673449.68,1,2020,11,3,1,4
8,-1,2,4.473383,4776184.28,7.923417,-1.367018,3,7.0,-4.7,-79603.071333,1189,42985658.52,0,2021,11,19,4,4
9,-1,4,1.002375,10849688.67,7.450955,30.147126,1,11.5,-3.6,4033.341513,345,75947820.69,0,2024,3,12,1,1


In [223]:
# Processed holdout Dataset to local
df_holdout.to_csv('Datasets/Processed_holdout_data.csv')
df_holdout.shape

(498, 17)

In [224]:
df_holdout.head(10)

,company_name,industry,funding_rounds,total_funding,job_postings_30d,employee_growth_pct,hiring_roles,industry_growth_rate,regional_employment_trend,funding_per_employee,days_since_last_funding,growth_momentum,funding_year,funding_month,funding_day,funding_weekday,funding_quarter
company_id,,,,,,,,,,,,,,,,,
COMP_000000,-1,3,7,4189415.46,9,-2.0,3,17.3,12.9,-41894.154600,1793,37704739.14,2020,3,25,2,1
COMP_000001,14332,3,1,3964281.97,5,20.1,1,8.8,-2.8,1878.806621,523,19821409.85,2023,9,16,5,3
COMP_000002,-1,0,2,7548375.72,9,7.0,2,15.9,-2.8,9435.469650,259,67935381.48,2024,6,6,3,2
COMP_000003,3412,0,3,8418569.83,4,39.7,3,17.5,2.3,2068.444676,1155,33674279.32,2021,12,23,3,4
COMP_000004,5732,1,2,2449746.19,13,34.2,0,27.0,4.4,695.950622,1769,31846700.47,2020,4,18,5,2
COMP_000005,-1,4,4,3463349.02,8,27.4,0,24.1,5.5,1219.489092,678,27706792.16,2023,4,14,4,2
COMP_000006,-1,4,2,2820737.83,15,38.1,1,26.2,8.1,721.416325,586,42311067.45,2023,7,15,5,3
COMP_000007,-1,2,3,1988942.70,13,46.9,0,22.1,6.4,415.228121,528,25856255.10,2023,9,11,0,3
COMP_000008,11329,4,4,6413583.09,10,1.5,2,25.3,-3.4,25654.332360,867,64135830.90,2022,10,7,4,4
